## TENSORFLOW - TENSORBOARD可视化

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

### tensorboard数据保存路径以及模型保存路径

In [13]:
tensorboard_dir = 'logs/mnist'   # 保存目录
if tf.io.gfile.exists(tensorboard_dir):
    tf.io.gfile.rmtree(tensorboard_dir)
tf.io.gfile.makedirs(tensorboard_dir)

model_dir = 'models/mnist' # 模型保存路径
if tf.io.gfile.exists(model_dir):
    tf.io.gfile.rmtree(model_dir)
tf.io.gfile.makedirs(model_dir)

### 载入数据

In [14]:
import os
from tensorflow.examples.tutorials.mnist import input_data
# 加载mnist数据
mnist_path = '/home/liujian/tensorflow_in_action/input/MNIST_data'
data = input_data.read_data_sets(mnist_path, one_hot=True)
# embeding降维可视化用的标签以及每个点的缩略图
LABELS = os.path.join(os.getcwd(), "resources/labels_1024.tsv")
SPRITES = os.path.join(os.getcwd(), "resources/sprite_1024.png")

Extracting /home/liujian/tensorflow_in_action/input/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/liujian/tensorflow_in_action/input/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/liujian/tensorflow_in_action/input/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/liujian/tensorflow_in_action/input/MNIST_data/t10k-labels-idx1-ubyte.gz


### 网络层

In [2]:
# 简单卷积层，为方便本章教程叙述，固定部分参数
def conv_layer(input,
               channels_in,    # 输入通道数
               channels_out,   # 输出通道数
              name='conv'):   # 名称 
    with tf.name_scope('name'):
        weights = tf.Variable(tf.truncated_normal([5, 5, channels_in, channels_out], stddev=0.05), name='W')
        biases = tf.Variable(tf.constant(0.05, shape=[channels_out]), name='B')
        conv = tf.nn.conv2d(input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')
        act = tf.nn.relu(conv + biases)

        tf.summary.histogram('weights', weights)
        tf.summary.histogram('biases', biases)
        tf.summary.histogram('activations', act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [3]:
# 简化全连接层
def fc_layer(input, num_inputs, num_outputs, name='fc'):
    with tf.name_scope(name):
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05), name='W')
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]), name='B')
        act = tf.matmul(input, weights) + biases
        
        tf.summary.histogram('weights', weights)
        tf.summary.histogram('biases', biases)
        tf.summary.histogram('activations', act)
        
        return act     

### 构建网络,计算交叉熵损失与准确率

In [12]:
def mnist_model(learning_rate, use_two_fc, use_two_conv, hparam):
    tf.reset_default_graph()    # 重置计算图
    sess = tf.Session()

    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    if use_two_conv:    # 是否使用两个卷积
        conv1 = conv_layer(x_image, 1, 32, "conv1")
        conv_out = conv_layer(conv1, 32, 64, "conv2")
    else:
        conv1 = conv_layer(x_image, 1, 64, "conv")    # 如果使用一个卷积，则再添加一个max_pooling保证维度相通
        conv_out = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    flatten_shape = conv_out.get_shape()[1:4].num_elements()
    flattened = tf.reshape(conv_out, [-1, flatten_shape])

    if use_two_fc:    # 是否使用两个全连接
        fc1 = fc_layer(flattened, flatten_shape, 1024, "fc1")
        relu = tf.nn.relu(fc1)
        embedding_input = relu
        tf.summary.histogram("fc1/relu", relu)
        embedding_size = 1024
        logits = fc_layer(fc1, 1024, 10, "fc2")
    else:
        embedding_input = flattened   # 新添加的embedding_input和embedding_size
        embedding_size = flatten_shape
        logits = fc_layer(flattened, flatten_shape, 10, "fc")

    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()    # 收集所有的summary

    # 添加embedding变量
    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()     # 保存训练过程

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(tensorboard_dir + hparam)
    writer.add_graph(sess.graph)

    # embedding的配置，详见官方文档
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = SPRITES
    embedding_config.metadata_path = LABELS
    # Specify the width and height of a single thumbnail.
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

    for i in range(1, 2001):
        batch = data.train.next_batch(100)
        if i % 100 == 0:    # 每100轮保存依存训练过程
            sess.run(assignment, feed_dict={x: data.test.images[:1024], y: data.test.labels[:1024]})
            run_options = tf.compat.v1.RunOptions(trace_level=tf.compat.v1.RunOptions.FULL_TRACE)
            run_metadata = tf.compat.v1.RunMetadata()
            train_accuracy, s = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]}, options=run_options, run_metadata=run_metadata)
            writer.add_summary(s, i)
            writer.add_run_metadata(run_metadata, 'step%03d' % i)
            print('Adding run metadata for', i)
            print("迭代轮次: {0:>6}, 训练准确率: {1:>6.4%}".format(i, train_accuracy))
            saver.save(sess, os.path.join(model_dir, "model.ckpt"), i)
        elif i % 10 == 0: # 每10轮写入一次
             # 同样，最好使用验证集
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)

        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

### 生成超参数的字符串

In [6]:
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

### 训练

In [11]:
for learning_rate in [1E-3, 1E-4, 1e-5]:
    for use_two_fc in [False, True]:
        for use_two_conv in [False, True]:
            hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
            print('Starting run for %s' % hparam)

            mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)

print('Done training!')

Starting run for lr_1E-03,conv=1,fc=1
Adding run metadata for 100
迭代轮次:    100, 训练准确率: 83.0000%
Adding run metadata for 200
迭代轮次:    200, 训练准确率: 93.0000%
Adding run metadata for 300
迭代轮次:    300, 训练准确率: 95.0000%
Adding run metadata for 400
迭代轮次:    400, 训练准确率: 96.0000%
Adding run metadata for 500
迭代轮次:    500, 训练准确率: 92.0000%
Adding run metadata for 600
迭代轮次:    600, 训练准确率: 97.0000%
Adding run metadata for 700
迭代轮次:    700, 训练准确率: 97.0000%
Adding run metadata for 800
迭代轮次:    800, 训练准确率: 97.0000%
Adding run metadata for 900
迭代轮次:    900, 训练准确率: 100.0000%
Adding run metadata for 1000
迭代轮次:   1000, 训练准确率: 97.0000%
Adding run metadata for 1100
迭代轮次:   1100, 训练准确率: 99.0000%
Adding run metadata for 1200
迭代轮次:   1200, 训练准确率: 99.0000%
Adding run metadata for 1300
迭代轮次:   1300, 训练准确率: 99.0000%
Adding run metadata for 1400
迭代轮次:   1400, 训练准确率: 95.0000%
Adding run metadata for 1500
迭代轮次:   1500, 训练准确率: 99.0000%
Adding run metadata for 1600
迭代轮次:   1600, 训练准确率: 96.0000%
Adding run metadata for 17

Adding run metadata for 1700
迭代轮次:   1700, 训练准确率: 99.0000%
Adding run metadata for 1800
迭代轮次:   1800, 训练准确率: 98.0000%
Adding run metadata for 1900
迭代轮次:   1900, 训练准确率: 99.0000%
Adding run metadata for 2000
迭代轮次:   2000, 训练准确率: 99.0000%
Starting run for lr_1E-04,conv=2,fc=2
Adding run metadata for 100
迭代轮次:    100, 训练准确率: 89.0000%
Adding run metadata for 200
迭代轮次:    200, 训练准确率: 90.0000%
Adding run metadata for 300
迭代轮次:    300, 训练准确率: 98.0000%
Adding run metadata for 400
迭代轮次:    400, 训练准确率: 95.0000%
Adding run metadata for 500
迭代轮次:    500, 训练准确率: 98.0000%
Adding run metadata for 600
迭代轮次:    600, 训练准确率: 95.0000%
Adding run metadata for 700
迭代轮次:    700, 训练准确率: 99.0000%
Adding run metadata for 800
迭代轮次:    800, 训练准确率: 99.0000%
Adding run metadata for 900
迭代轮次:    900, 训练准确率: 99.0000%
Adding run metadata for 1000
迭代轮次:   1000, 训练准确率: 97.0000%
Adding run metadata for 1100
迭代轮次:   1100, 训练准确率: 99.0000%
Adding run metadata for 1200
迭代轮次:   1200, 训练准确率: 98.0000%
Adding run metadata for 130